# Knowledge Store 使用指南

# 1. 初始化


In [2]:
from tests.conftest import MockEmbedding
from tablestore_for_agent_memory.knowledge.knowledge_store import KnowledgeStore
import tablestore
import os

endpoint = os.getenv("tablestore_end_point")
instance_name = os.getenv("tablestore_instance_name")
access_key_id = os.getenv("tablestore_access_key_id")
access_key_secret = os.getenv("tablestore_access_key_secret")

# 创建 tablestore 的 sdk client
tablestore_client = tablestore.OTSClient(
    endpoint,
    access_key_id,
    access_key_secret,
    instance_name,
    retry_policy=tablestore.WriteRetryPolicy(),
)

# 需要索引哪些meta字段，下面进行举例。document_id/tenant_id/text_content/vector_content这些必备字段不需要设置。
search_index_schema = [
    tablestore.FieldSchema("user_id", tablestore.FieldType.KEYWORD),
    tablestore.FieldSchema("meta_string", tablestore.FieldType.KEYWORD),
    tablestore.FieldSchema("meta_long", tablestore.FieldType.LONG),
    tablestore.FieldSchema("meta_double", tablestore.FieldType.DOUBLE),
    tablestore.FieldSchema("meta_boolean", tablestore.FieldType.BOOLEAN),
]

# 假设我们使用的embedding模型是维度=4
embedding_model = MockEmbedding(4)

knowledge_store = KnowledgeStore(
    tablestore_client=tablestore_client,
    vector_dimension=4,  # 需要与embedding模型的维度相等，上述我们使用的embedding模型维度为4。大多数生产使用的embedding模型维度为512/768等
    enable_multi_tenant=True,  # 是否开启多租户能力，这里以开启多租户为例。
    search_index_schema=search_index_schema,
)

### 多租户能力说明
在多租户场景中，租户可以是知识库、用户、组织等，具体可以参考业务场景。通常来说使用用户id或者知识库id当做租户id有通用性。

- 多租户场景举例：以用户id为例，该数据库中存储了不同用户的数据，在每次查询时候，一定会带上用户id的filter限制，来查询这个用户的数据里最相似的文档，则该场景是适合多租户能力的。开启多租户后，会在多元索引里设置路由，同时开启一些向量检索的多租户优化能力，降低查询延时，提高召回率。
- 非多租户场景举例：数据库中只存储了一个用户的某一个领域的全部数据，查询时候，不会添加用户id、知识库id等租户特征的filter条件，每次都是在全部数据中查找最相似的文档，该场景则适合非多租户。




In [14]:
# 创建表(包括多元索引)
# 仅需执行一次
knowledge_store.init_table()

In [ ]:
# 删除表（内部接口，仅供测试使用）
knowledge_store._delete_table()

 # 2. 文档管理

本章节主要介绍文档的增删改查。

## 2.1 声明 Document

In [23]:
from tablestore_for_agent_memory.base.base_knowledge_store import Document

# 声明一个Document
# document_id和tenant_id(租户id)唯一确认一行数据。 如果创建 KnowledgeStore 时候没开启多租户能力，tenant_id 参数不填即可。
document = Document(document_id="1", tenant_id="user_id_1")
# 文档的文本内容
document.text_content = "123"
# 文档的向量内容，格式为 list[float], 此处我们调用embedding模型将文档的文本内容转成向量
document.vector_content = embedding_model.embedding(document.text_content)
# 声明一些额外的meta信息
document.metadata["meta_string"] = "123"
document.metadata["meta_bool"] = True
document.metadata["meta_double"] = 123.456
document.metadata["meta_long"] = 123456
document.metadata["meta_bytes"] = bytearray("China", encoding="utf8")

## 2.2 创建、更新、查询、删除 Session


In [24]:
# 创建(不管是否存在，都会进行覆盖写入)
knowledge_store.put_document(document)
# 查询 （如果创建 KnowledgeStore 时候没开启多租户能力，tenant_id 参数不填即可。）
document = knowledge_store.get_document(document_id="1", tenant_id="user_id_1")
print(document)
print(document.metadata["meta_string"])

Document(document_id='1', tenant_id='user_id_1', text_content='123', vector_content=[0.24378446113768007, 0.24378446113768007, 0.24378446113768007, 0.24378446113768007], metadata={'meta_bool': True, 'meta_bytes': bytearray(b'China'), 'meta_double': 123.456, 'meta_long': 123456, 'meta_string': '123'})
123


In [30]:
document_for_update = Document(document_id="1", tenant_id="user_id_1")
document_for_update.metadata["meta_string"] = "updated from 123"
# 增量更新 (无更新场景，推荐使用 put_document 进行覆盖写入) 此处我们将上述文档的"meta_string"字段修改为一个新值
knowledge_store.update_document(document_for_update)
# 再次查询该文档，得到更新后的值
document = knowledge_store.get_document(document_id="1", tenant_id="user_id_1")
print(document.metadata["meta_string"])

updated from 123


In [31]:
# 批量获取文档
id_list = ["0", "1", "2"]
# 如果创建 KnowledgeStore 时候没开启多租户能力，tenant_id 参数不填即可。
documents_by_batch_get = knowledge_store.get_documents(document_id_list=id_list, tenant_id="user_id_1")
print("第0个文档:", documents_by_batch_get[0])
print("第1个文档:", documents_by_batch_get[1])
print("第2个文档:", documents_by_batch_get[2])

第0个文档: None
第1个文档: Document(document_id='1', tenant_id='user_id_1', text_content='123', vector_content=[0.24680628233411753, 0.24680628233411753, 0.24680628233411753, 0.24680628233411753], metadata={'meta_bool': True, 'meta_bytes': bytearray(b'China'), 'meta_double': 123.456, 'meta_long': 123456, 'meta_string': 'updated from 123'})
第2个文档: None


In [32]:
# 删除（如果创建 KnowledgeStore 时候没开启多租户能力，tenant_id 参数不填即可。）
knowledge_store.delete_document(document_id="1", tenant_id="user_id_1")
print(knowledge_store.get_document(document_id="1", tenant_id="user_id_1"))

None


In [25]:
# 根据租户删除数据
knowledge_store.delete_document_by_tenant(tenant_id="user_id_1")
print(knowledge_store.get_document(document_id="1", tenant_id="user_id_1"))

None


## 2.3 搜索文档

### 2.3.1 写入样例数据
 

In [33]:
from faker import Faker
import random

fk_data = Faker(locale="zh_CN")


def random_document() -> Document:
    tenant_id = random.choice(["1", "2"])
    document = Document(document_id=fk_data.uuid4(), tenant_id=tenant_id)
    # 文档的文本内容
    document.text_content = " ".join(
        random.choices(
            ["abc", "def", "ghi", "abcd", "adef", "abcgh", "apple", "banana", "cherry"], k=random.randint(1, 10)
        )
    )
    # 文档的向量内容，格式为 list[float], 此处我们调用embedding模型将文档的文本内容转成向量
    document.vector_content = embedding_model.embedding(document.text_content)
    # 声明一些额外的meta信息
    document.metadata["meta_string"] = fk_data.name_female()
    document.metadata["meta_long"] = random.randint(0, 88888888)
    document.metadata["meta_double"] = random.uniform(1.0, 2.0)
    document.metadata["meta_boolean"] = random.choice([False, True])
    document.metadata["meta_bytes"] = bytearray(fk_data.city_name(), encoding="utf8")
    return document


total_count = 100
for i in range(total_count):
    document = random_document()
    knowledge_store.put_document(document)

### 2.3.2 向量检索


In [10]:
from tablestore_for_agent_memory.base.filter import Filters

# 查询和“abc”相似的文档
query_vector = embedding_model.embedding("abc")

document_hits, _ = knowledge_store.vector_search(
    query_vector=query_vector,
    top_k=20,
    tenant_id="1",  # 租户id。(如果创建 KnowledgeStore 时候没开启多租户能力，tenant_id 参数不填即可)
    metadata_filter=Filters.logical_and(
        [Filters.eq("meta_boolean", True), Filters.gt("meta_long", 1)]
    ),  # 添加过滤条件：meta_boolean=true 且 meta_long>1
)
print(len(document_hits))

# 展示前2个文档
for hit in document_hits[:2]:
    print("文档内容:", hit.document)
    print("文档分数:", hit.score)

20
文档内容: Document(document_id='1f69fed7-b195-40dc-b2d0-5c6c4bb662b7', tenant_id='1', text_content='apple', vector_content=None, metadata={'meta_boolean': True, 'meta_double': 1.5448139375851748, 'meta_long': 17396403, 'meta_string': '罗桂芝'})
文档分数: 1.0
文档内容: Document(document_id='3960a0e9-5565-41bd-ab43-158baf9f2596', tenant_id='1', text_content='ghi abcd', vector_content=None, metadata={'meta_boolean': True, 'meta_double': 1.131841860692934, 'meta_long': 47111749, 'meta_string': '张玉华'})
文档分数: 1.0


### 2.3.3 全文检索

In [11]:
# 全文检索“abc”相关的文档

# next_token可以进行连续翻页
document_hits, next_token = knowledge_store.full_text_search(
    query="abc",
    tenant_id="1",  # 租户id。(如果创建 KnowledgeStore 时候没开启多租户能力，tenant_id 参数不填即可)
    limit=20,
    metadata_filter=Filters.logical_and(
        [Filters.eq("meta_boolean", True), Filters.gt("meta_long", 1)]
    ),  # 添加过滤条件：meta_boolean=true 且 meta_long>1
)

print(len(document_hits))

# 展示前2个文档
for hit in document_hits[:2]:
    print("文档内容:", hit.document)
    print("文档分数:", hit.score)

9
文档内容: Document(document_id='db266e1a-76e8-418d-8824-d4044a47471f', tenant_id='1', text_content='apple apple abc abc banana', vector_content=None, metadata={'meta_boolean': True, 'meta_double': 1.3339972044754749, 'meta_long': 76492008, 'meta_string': '王坤'})
文档分数: 3.535884380340576
文档内容: Document(document_id='6778137e-29f8-4e25-887f-ae4336c9550b', tenant_id='1', text_content='def abc', vector_content=None, metadata={'meta_boolean': True, 'meta_double': 1.746714507976319, 'meta_long': 11274797, 'meta_string': '王成'})
文档分数: 3.4931881427764893


### 2.3.4 通用检索



In [18]:
document_hits, next_token = knowledge_store.search_documents(
    tenant_id="1",  # 租户id。(如果创建 KnowledgeStore 时候没开启多租户能力，tenant_id 参数不填即可)
    limit=20,
    metadata_filter=Filters.logical_and(
        [Filters.eq("meta_boolean", True), Filters.gt("meta_long", 1)]
    ),  # 添加过滤条件：meta_boolean=true 且 meta_long>1
    meta_data_to_get=["text_content", "meta_string", "meta_boolean"],  # 只返回这三个字段,不填则默认返回索引里有的字段
)

print(len(document_hits))

# 展示前2个文档
for hit in document_hits[:2]:
    print("文档内容:", hit.document)
    print("文档分数:", hit.score)

20
文档内容: Document(document_id='1f69fed7-b195-40dc-b2d0-5c6c4bb662b7', tenant_id='1', text_content='apple', vector_content=None, metadata={'meta_boolean': True, 'meta_string': '罗桂芝'})
文档分数: None
文档内容: Document(document_id='3960a0e9-5565-41bd-ab43-158baf9f2596', tenant_id='1', text_content='ghi abcd', vector_content=None, metadata={'meta_boolean': True, 'meta_string': '张玉华'})
文档分数: None


In [19]:
# 翻页获取所有数据, 每页获取limit=3.
total_document_hits = []
next_token = None
while True:
    document_hits, next_token = knowledge_store.search_documents(
        tenant_id="1",
        limit=3,
        meta_data_to_get=[
            "text_content",
            "meta_string",
            "meta_boolean",
        ],  # 只返回这三个字段,不填则默认返回索引里有的字段
        next_token=next_token,
    )
    total_document_hits.extend(document_hits)
    if next_token is None:
        break
print("total docs", len(total_document_hits))

total docs 55
